## Objective
Parses a text file table that has reinsurance and risk adjustment data for a benefit year into a CSV.

### Preprocessing
step 1: copy into raw text file

step 2: delete header 

HIOS ID
HIOS INPUTTED
INSURANCE COMPANY
NAME STATE
REINSURANCE
PAYMENT
AMOUNT (OR
NOT ELIGIBLE)
HHS RISK
ADJUSTMENT
TRANSFER
AMOUNT
(INDIVIDUAL
MARKET,
INCLUDING
CATASTROPHIC)
HHS RISK
ADJUSTMENT
TRANSFERS
AMOUNT (SMALL
GROUP MARKET)

step 3: delete page numbers through the regexp ^RI RA.\*

step 4: remove blank lines through the regexp ^\n\n

step 5: find and replace "not eligible" with "N/E"

step 6: find and replace "\$ -" with "\$-"

step 7: find and replace "\$ " with "\$" (also works for step 6)

step 8: find and replace "N/A - MA Issuer" with "N/A_MA_Issuer" (dataframe has trouble parsing "-")

step 9: find and replace "N/A - Default Charge" with "N/A_DefaultCharge"

step 10: delete footnotes on the first page

In [1]:
import pandas as pd
# import re

In [2]:
#with open('2014-Benefit-Year-Edit.txt', 'r') as myfile:
with open('2017pdf.txt', 'r') as myfile:
    s = myfile.read()

In [3]:
split = str.split(s)
length = len(split)

In [4]:
storage = pd.DataFrame(columns=["HIOS ID", "HIOS INPUTTED INSURANCE COMPANY NAME", "STATE",
                                "HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL NONCATASTROPHIC RISK POOL)",
                                "HHS RISK ADJUSTMENT TRANSFER AMOUNT (CATASTROPHIC RISK POOL)",
                                "HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP RISK POOL)"])

# prog = re.compile('^[A-Z]{2}$')

i = 0
index = 0
while i < length - 1:
    hios_id = split[i]
    i = i + 1
    temp = i
    # while (prog.match(split[i]) == None or split[i] in ['US', 'BC', 'OF']): # look for state in quotes
    while (split[i][:1] != '(' and split[i][:1] != '$'):
        i = i + 1
    company_name = str(' '.join(split[temp:i - 1]))
    state = split[i - 1]
    reinsurance = split[i]
    i = i + 1
    individual = split[i]
    i = i + 1
    small_group = split[i] 
    i = i + 1
    storage.loc[index] = [hios_id, company_name, state, reinsurance, individual, small_group]
    index = index + 1
    
storage.head()

,HIOS ID,HIOS INPUTTED INSURANCE COMPANY NAME,STATE,HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL NONCATASTROPHIC RISK POOL),HHS RISK ADJUSTMENT TRANSFER AMOUNT (CATASTROPHIC RISK POOL),HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP RISK POOL)
0,11082,"Aetna, Inc.",AK,$-,$-,"$70,609.53"
1,38344,Premera Blue Cross,AK,$0.00,$-,"$1,615,253.34"
2,73836,"Moda Health Plan, Inc.",AK,$-,$-,"($744,336.01)"
3,80049,"UnitedHealth Group, Inc.",AK,$-,$-,"($941,526.90)"
4,46944,Blue Cross Blue Shield of Alabama,AL,"($172,651.78)",$0.00,"$1,662,986.37"


In [5]:
# save it
storage.to_csv("Output/2017-PDF.csv")

In [6]:
storage.tail()

,HIOS ID,HIOS INPUTTED INSURANCE COMPANY NAME,STATE,HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL NONCATASTROPHIC RISK POOL),HHS RISK ADJUSTMENT TRANSFER AMOUNT (CATASTROPHIC RISK POOL),HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP RISK POOL)
605,95628,"UnitedHealth Group, Inc.",WV,$-,$-,"($240,716.99)"
606,11269,Blue Cross Blue Shield of Wyoming,WY,$0.00,$0.00,"($623,916.29)"
607,44325,"Aetna, Inc.",WY,$-,$-,"$244,027.31"
608,49714,"UnitedHealth Group, Inc.",WY,$-,$-,"$308,081.87"
609,79022,"Aetna, Inc.",WY,$-,$-,"$71,807.08"
